# Multivariate Logistic Regression using hybrid MM-initial + Newton's for joint estimation
    
   ### Simulation Parameters (Truth):
   $$p = 0.300, vc_1 = 0.100, vc_2 = 0.100$$
    
    
       1. Using initial MM + Newton + mehrotra: (59 iterations, 102.6 seconds)
   $$\hat p = 0.301, \hat vc_1 = 0.082, \hat vc_2 = 0.080$$
    

In [1]:
using GLMCopula, Random, Statistics, Test, LinearAlgebra, StatsFuns, GLM, Revise

In [2]:
Random.seed!(1234)
n = 50
variance_component_1 = 0.1
variance_component_2 = 0.1
Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)

mean = 0.3
dist = Bernoulli
vecd = [dist(mean) for i in 1:n]
    
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

NonMixedMultivariateDistribution{Discrete,Bernoulli{Float64},Array{Bernoulli{Float64},1}}(Bernoulli{Float64}[Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3)  …  Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3), Bernoulli{Float64}(p=0.3)], [0.2 0.1 … 0.1 0.1; 0.1 0.2 … 0.1 0.1; … ; 0.1 0.1 … 0.2 0.1; 0.1 0.1 … 0.1 0.2], 9.999999999999996, Union{ContinuousUnivariateCopula, DiscreteUnivariateCopula}[#undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef  …  #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef])

In [3]:
nsample = 10_000
Random.seed!(1234)
@time Y_nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, nsample)

  3.849030 seconds (9.72 M allocations: 7.164 GiB, 22.33% gc time)


10000-element Array{Array{Float64,1},1}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0  …  1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
 [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0  …  0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
 [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
 [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0  …  1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
 [0.0, 0.0, 1.

In [4]:
d = Bernoulli()
link = LogitLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, nsample)
for i in 1:nsample
    y = Float64.(Y_nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -308675.8817541355 39999
2 -308675.8817541355 -308675.8817541355 39999
initializing variance components using MM-Algorithm
gcm.β = [-0.8102932942177399]


2-element Array{Float64,1}:
 0.027230981358980718
 0.0008757048338999331

In [5]:
loglikelihood!(gcm, true, true)

-307479.5042351658

## Newton using Hessian + mehrotra

In [6]:
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))

gcm.θ = [-0.8102932942177399, 0.027230981358980718, 0.0008757048338999331]
gcm.∇θ = [-1362.849972613486, -3.0923377111030277, -142.5346354638168]
gcm.θ = [-0.8102932942177399, 9.99999999, 9.99999999]
gcm.∇θ = [-2297.7997286655273, -28.973193191628525, 28.846929497851306]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total

  29  3.0746096e+05 0.00e+00 7.31e-03 -11.0 3.29e-05    -  1.00e+00 1.00e+00f  1
gcm.θ = [-0.8403350973395807, 0.08187760300587918, 0.07947448478200463]
gcm.∇θ = [-0.06319747395686282, 0.008388990347157232, -0.0018892716034315526]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  3.0746096e+05 0.00e+00 4.64e-03 -11.0 2.08e-05    -  1.00e+00 1.00e+00f  1
gcm.θ = [-0.8403360005847587, 0.08188676703565084, 0.0794876970880304]
gcm.∇θ = [-0.040040872572808084, 0.005318955649272539, -0.0011982557844021358]
  31  3.0746096e+05 0.00e+00 2.94e-03 -11.0 1.32e-05    -  1.00e+00 1.00e+00f  1
gcm.θ = [-0.8403365728657423, 0.08189257954900478, 0.07949607731784016]
gcm.∇θ = [-0.025390238194049175, 0.0033692005381453782, -0.0007591674220748246]
  32  3.0746096e+05 0.00e+00 1.86e-03 -11.0 8.38e-06    -  1.00e+00 1.00e+00f  1
gcm.θ = [-0.8403369357536207, 0.08189626225262422, 0.07950138687901523]
gcm.∇θ = [-0.016086325721849093, 0.002136113922460936, -0.0004813834020

-307460.961986127

In [7]:
@show GLMCopula.loglikelihood!(gcm, true, true);

GLMCopula.loglikelihood!(gcm, true, true) = -307460.961986127


In [8]:
@show gcm.θ;

gcm.θ = [-0.8403375636364836, 0.08190263795116795, 0.07951057908020588]


In [9]:
@show gcm.∇θ;

gcm.∇θ = [-1.0564409080870973e-7, 1.5148543752729893e-8, -5.042238626629114e-9]


In [11]:
println("estimated mean = $(exp(gcm.β[1]) / (1 + exp(gcm.β[1]))); true mean value= $mean")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2");

estimated mean = 0.3014636939573715; true mean value= 0.3
estimated variance component 1 = 0.08190263795116795; true variance component 1 = 0.1
estimated variance component 2 = 0.07951057908020588; true variance component 2 = 0.1
